# Local Flood Mapping

In [ ]:
from dask_flood_mapper import flood

In [ ]:
from dask.distributed import Client

client = Client(processes=False, threads_per_worker=2, n_workers=1, memory_limit="20GB")

In [ ]:
time_range = "2022-10-11/2022-10-25"
bbox = [12.3, 54.3, 13.1, 54.6]
fd = flood.decision(bbox=bbox, datetime=time_range).compute()
fd

In [ ]:
from importlib.resources import files
import xarray as xr

data_text = files("dask_flood_mapper.data").joinpath("wcover.tif")
wcover = xr.open_dataarray(data_text)

In [ ]:
from bokeh.models import FixedTicker
import hvplot.xarray

fd = fd.where(wcover != 80)
colorbar_opts = {
    "major_label_overrides": {
        0: "non-flood",
        1: "flood",
    },
    "ticker": FixedTicker(ticks=[0, 1]),
}
fd.hvplot.quadmesh(
    x="x",
    y="y",
    rasterize=True,
    geo=True,
    cmap=["rgba(0, 0, 1, 0.1)", "darkred"],
).opts(frame_height=400, colorbar_opts={**colorbar_opts})

In [ ]:
client.close()